In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)


/home/jac/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


file path ../features/cnn_glove_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_glove_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_gru_glove_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_gru_glove_1_trainable_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_1_feat_de_fr.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_muse_adj_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_glove_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/gru_muse_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_2_feat.pkl
(95851, 6) (226998, 6)
fi

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

input_len = train_x.shape[1]
num_classes = 6

def get_nn_model():
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(input_len,)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    return model

tmp_m = get_nn_model()
tmp_m.summary()
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 256)               61184     
_________________________________________________________________
dense_56 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_57 (Dense)             (None, 6)                 774       
Total params: 94,854
Trainable params: 94,854
Non-trainable params: 0
_________________________________________________________________


In [36]:
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau,EarlyStopping
from keras.models import load_model
from keras.optimizers import Adam

def lr_s(e):
    if e < 5:
        return 0.001
    elif e < 10:
        return 0.0001
    else:
        return 0.00001
    

def nn_eval(y_true,y_pred):
    res = []
    for i in range(6):
        a = y_true[:,i]
        b = y_pred[:,i]
        res.append(log_loss(a,b))
    return res

def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0

        model = get_nn_model()
        model.compile(loss='binary_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
        
        file_path = 'nn_ens.h5'
        chk = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                                patience=3,
                                                verbose=1,
                                                factor=0.5,
                                                min_lr=0.000001)
        
        early = EarlyStopping(monitor='val_loss', patience=10)
        
        model.fit(curr_x,curr_y,
                  batch_size=128,
                  epochs=100,
                  validation_data=(hold_out_x,hold_out_y),
                  callbacks=[chk,learning_rate_reduction,early]
                 )
        model = load_model(file_path)
        
        curr_tr_pred = model.predict(curr_x)
        curr_val_pred = model.predict(hold_out_x)
        curr_test_pred = model.predict(test_x)
        test_pred += curr_test_pred
        
        tr_res = nn_eval(curr_y,curr_tr_pred)
        val_res = nn_eval(hold_out_y,curr_val_pred)
        train_loss_l = np.mean(tr_res)
        val_loss_l = np.mean(val_res)
        print(tr_res,train_loss_l)
        print(val_res,val_loss_l)
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k

        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')

done


In [37]:
xgb_res = simple_ens('xgb',k=5)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/nn_ens_new_csv_fold5.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')




Train on 76680 samples, validate on 19171 samples
Epoch 1/100
76680/76680 [==============================] - 3s 39us/step - loss: 0.1601 - acc: 0.9763 - val_loss: 0.1288 - val_acc: 0.9786
Epoch 2/100
76680/76680 [==============================] - 2s 27us/step - loss: 0.0906 - acc: 0.9808 - val_loss: 0.0638 - val_acc: 0.9825
Epoch 3/100
76680/76680 [==============================] - 2s 26us/step - loss: 0.0606 - acc: 0.9824 - val_loss: 0.0588 - val_acc: 0.9804
Epoch 4/100
76680/76680 [==============================] - 2s 26us/step - loss: 0.0523 - acc: 0.9833 - val_loss: 0.0536 - val_acc: 0.9837
Epoch 5/100
76680/76680 [==============================] - 2s 27us/step - loss: 0.0489 - acc: 0.9835 - val_loss: 0.0483 - val_acc: 0.9842
Epoch 6/100
76680/76680 [==============================] - 2s 27us/step - loss: 0.0464 - acc: 0.9837 - val_loss: 0.0457 - val_acc: 0.9842
Epoch 7/100
76680/76680 [==============================] - 2s 27us/step - loss: 0.0450 - acc: 0.9839 - val_loss: 0.0422 - 

/home/jac/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/jac/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1694: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[nan, 0.019479271746140725, nan, 0.0069934080193730947, 0.054630047234466232, 0.017970996869438958] nan
[nan, 0.024324997959981706, nan, 0.0091981501719756928, 0.055430236062574835, 0.0208025882635689] nan
Train on 76681 samples, validate on 19170 samples
Epoch 1/100
76681/76681 [==============================] - 3s 43us/step - loss: 0.1468 - acc: 0.9770 - val_loss: 0.0876 - val_acc: 0.9809
Epoch 2/100
76681/76681 [==============================] - 2s 26us/step - loss: 0.0774 - acc: 0.9817 - val_loss: 0.0839 - val_acc: 0.9767
Epoch 3/100
76681/76681 [==============================] - 2s 26us/step - loss: 0.0605 - acc: 0.9827 - val_loss: 0.0566 - val_acc: 0.9818
Epoch 4/100
76681/76681 [==============================] - 2s 25us/step - loss: 0.0549 - acc: 0.9830 - val_loss: 0.0487 - val_acc: 0.9834
Epoch 5/100
76681/76681 [==============================] - 2s 26us/step - loss: 0.0496 - acc: 0.9836 - val_loss: 0.0470 - val_acc: 0.9837
Epoch 6/100
76681/76681 [=============================